# Speed up analysis of caches, also save box vectors

In [1]:
import math
from simtk import unit
import os
import tempfile
import pickle
import mdtraj as md
import numpy as np
from simtk.unit.quantity import Quantity
import logging 

# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

from matplotlib import pyplot as plt
from simtk.openmm import app
from tqdm import tqdm, tqdm_notebook
import argparse
import random
import time


In [2]:
def get_dihedrals(i, name, length, out_dir, htf, dihedral_indices_new, dihedral_indices_old):
    
    from tqdm import tqdm
    
    old_top = md.Topology.from_openmm(htf._topology_proposal.old_topology)
    new_top = md.Topology.from_openmm(htf._topology_proposal.new_topology)
    
    # Load nc file
    # From Hannah: https://github.com/hannahbrucemacdonald/endstate_pdbs/blob/master/scripts/input_for_pol_calc.py
    from perses.analysis.utils import open_netcdf
    nc = open_netcdf(os.path.join(out_dir, f"{i}_{phase}_{name.lower()}_{length}ns.nc"))
    nc_checkpoint = open_netcdf(os.path.join(out_dir, f"{i}_{phase}_{name.lower()}_{length}ns_checkpoint.nc"))
    checkpoint_interval = nc_checkpoint.CheckpointInterval
    all_positions = nc_checkpoint.variables['positions']
    n_iter, n_replicas, n_atoms, _ = np.shape(all_positions)
    n_iter = 10

    # Initialize arrays for old, new, and hybrid positions
    all_pos_new = np.zeros(shape=(n_iter, new_top.n_atoms, 3))
    all_pos_old = np.zeros(shape=(n_iter, old_top.n_atoms, 3))
    all_pos_hybrid = np.zeros(shape=(n_iter, n_atoms, 3)) # includes solvent atoms
    
    def new_positions(htf, hybrid_positions):
        n_atoms_new = htf._topology_proposal.n_atoms_new
        hybrid_indices = [htf._new_to_hybrid_map[idx] for idx in range(n_atoms_new)]
        print("shape ", hybrid_positions.shape)
        return hybrid_positions[hybrid_indices, :]
    
    def old_positions(htf, hybrid_positions):
        n_atoms_old = htf._topology_proposal.n_atoms_old
        hybrid_indices = [htf._old_to_hybrid_map[idx] for idx in range(n_atoms_old)]
        return hybrid_positions[hybrid_indices, :]
    
    # Save old, new, and hybrid positions
    get_time = 0
    old_time = 0
    new_time = 0
    hybrid_time = 0
    for iteration in tqdm(range(n_iter)):
        replica_id = 0
        pos = all_positions[iteration,replica_id,:,:] *unit.nanometers
        print("shape ", pos.shape)
        all_pos_new[iteration] = new_positions(htf, pos).value_in_unit_system(unit.md_unit_system) # Get new positions only
        all_pos_hybrid[iteration] = pos # Get hybrid positions
        all_pos_old[iteration] = old_positions(htf, pos).value_in_unit_system(unit.md_unit_system)

#         initial_time = time.time()
#         replica_id = np.where(nc.variables['states'][iteration*checkpoint_interval] == 0)[0]
#         pos = all_positions[iteration,replica_id,:,:][0] *unit.nanometers
#         elapsed_time = (time.time() - initial_time)
#         get_time += elapsed_time

#         initial_time = time.time()
#         all_pos_new[iteration] = new_positions(pos).value_in_unit_system(unit.md_unit_system) 
#         elapsed_time = (time.time() - initial_time)
#         new_time += elapsed_time
        
#         initial_time = time.time()
#         all_pos_hybrid[iteration] = pos 
#         elapsed_time = (time.time() - initial_time)
#         hybrid_time += elapsed_time
        
#         initial_time = time.time()
#         all_pos_old[iteration] = old_positions(pos).value_in_unit_system(unit.md_unit_system)
#         elapsed_time = (time.time() - initial_time)
#         old_time += elapsed_time
    
#     print(f"get time {get_time}") 
#     print(f"old time: {old_time}")
#     print(f"new time: {new_time}")
#     print(f"hybrid time: {hybrid_time}")
    
    dihedrals_all = []
    for pos, top, indices in zip([all_pos_new, all_pos_old], [new_top, old_top], [dihedral_indices_new, dihedral_indices_old]):
        traj = md.Trajectory(np.array(pos), top)
        dihedrals = md.compute_dihedrals(traj, np.array([indices]))
        dihedrals_all.append(dihedrals)
    
    return dihedrals_all, n_iter, all_pos_hybrid
#     traj = traj.image_molecules()
    
#     traj.save("test_old.dcd")
#     traj[0].save("test_old.pdb")
    
#     return dihedrals_all, n_iter, all_pos_hybrid
    
def plot_dihedrals(dihedrals, outfile):
    # Plot histogram with error bars : https://stackoverflow.com/questions/35390276/how-to-add-error-bars-to-histogram-diagram-in-python
    entries, edges, _ = plt.hist(dihedrals)
    bin_centers = 0.5 * (edges[:-1] + edges[1:]) # calculate bin centers
    plt.errorbar(bin_centers, entries, yerr=np.sqrt(entries), fmt='r.') # draw errobars, use the sqrt error
    plt.xlim(-np.pi, np.pi)
    plt.savefig(outfile, dpi=300)
    plt.close()
    
def plot_time_series(dihedrals, n_iter, outfile):
    from perses.dispersed import feptasks
    t0, g, neff_max, a_t, uncorrelated_indices = feptasks.compute_timeseries(dihedrals)

    plt.scatter(range(n_iter), dihedrals)
    plt.ylabel("dihedral")
    plt.xlabel("iteration number")
    plt.ylim(-np.pi, np.pi)
    plt.savefig(outfile, dpi=300)
    plt.close()
    
    return uncorrelated_indices
    
def plot_dihedrals_uncorrelated(dihedrals, uncorrelated_indices, outfile):
    # Plot histogram with error bars : https://stackoverflow.com/questions/35390276/how-to-add-error-bars-to-histogram-diagram-in-python
    entries, edges, _ = plt.hist(dihedrals[uncorrelated_indices])
    bin_centers = 0.5 * (edges[:-1] + edges[1:]) # calculate bin centers
    plt.errorbar(bin_centers, entries, yerr=np.sqrt(entries), fmt='r.') # draw errobars, use the sqrt error
    plt.xlim(-np.pi, np.pi)
    plt.savefig(outfile, dpi=300)
    plt.close()

In [3]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/14/64/"
endstate = 0
phase = "apo"
length = 1
i = 64
old_aa_name = 'ASN'
new_aa_name = 'TYR'
resid = '501'
name = 'asn'

In [4]:
with open(os.path.join(outdir, f"{i}_{phase}_{endstate}.pickle"), 'rb') as f:
    htf = pickle.load(f)


In [7]:
from perses.analysis.utils import open_netcdf
nc = open_netcdf(os.path.join(outdir, f"{i}_{phase}_{name.lower()}_{length}ns.nc"))
nc_checkpoint = open_netcdf(os.path.join(outdir, f"{i}_{phase}_{name.lower()}_{length}ns_checkpoint.nc"))

In [18]:
np.array(nc_checkpoint['box_vectors']).shape

(1001, 1, 3, 3)

In [20]:
np.array(nc_checkpoint['box_vectors'])[0]

array([[[ 8.692843 ,  0.       ,  0.       ],
        [-2.8976142,  8.195692 ,  0.       ],
        [-2.8976142, -4.097845 ,  7.097677 ]]], dtype=float32)

In [19]:
with open(os.path.join(outdir, "box_vectors.npy"), "wb") as f:
    np.save(f, np.array(nc_checkpoint['box_vectors']))


In [29]:
thr_dihedral = ['N', 'CA', 'CB', 'OG1']
other_dihedral = ['N', 'CA', 'CB', 'CG']
ala_dihedral = ['N', 'CA', 'CB', 'HB1']
asp_dihedral = ['CA', 'CB', 'CG', 'OD2']
ile_dihedral = ['N', 'CA', 'CB', 'CG2']

dihedral_atoms = []
for aa_name in [old_aa_name, new_aa_name]:
    if aa_name in ["PHE", "TYR", "TRP", "GLU", "LYS", "ARG", "GLN", "ASN"]:
        dihedral_atoms.append(other_dihedral)
    elif aa_name == "THR":
        dihedral_atoms.append(thr_dihedral)
    elif aa_name == "ALA":
        dihedral_atoms.append(ala_dihedral)
    elif aa_name == 'ASP':
        dihedral_atoms.append(asp_dihedral)
    elif aa_name == 'ILE':
        dihedral_atoms.append(ile_dihedral)
        

for res in htf._topology_proposal.old_topology.residues():
    if res.id == resid and res.chain.index == 0:
        residue_old = res
for res in htf._topology_proposal.new_topology.residues():
    if res.id == resid and res.chain.index == 0:
        residue_new = res
# indices_old = [atom.index for atom in list(htf._topology_proposal.old_topology.residues())[int(args.resid)].atoms() if atom.name in dihedral_atoms[0]]
# indices_new = [atom.index for atom in list(htf._topology_proposal.new_topology.residues())[int(args.resid)].atoms() if atom.name in dihedral_atoms[1]]
indices_old = [atom.index for atom in residue_old.atoms() if atom.name in dihedral_atoms[0]]
indices_new = [atom.index for atom in residue_new.atoms() if atom.name in dihedral_atoms[1]]
_logger.info(f"old indices: {indices_old}")
_logger.info(f"new indices: {indices_new}")



INFO:root:old indices: [2605, 2607, 2609, 2612]
INFO:root:new indices: [2605, 2607, 2609, 2612]


In [30]:
for atom in htf._topology_proposal.new_topology.atoms():
    if atom.residue.id == '501':
        print(atom)

<Atom 2605 (N) of chain 0 residue 169 (TYR)>
<Atom 2606 (H) of chain 0 residue 169 (TYR)>
<Atom 2607 (CA) of chain 0 residue 169 (TYR)>
<Atom 2608 (HA) of chain 0 residue 169 (TYR)>
<Atom 2609 (CB) of chain 0 residue 169 (TYR)>
<Atom 2610 (HB2) of chain 0 residue 169 (TYR)>
<Atom 2611 (HB3) of chain 0 residue 169 (TYR)>
<Atom 2612 (CG) of chain 0 residue 169 (TYR)>
<Atom 2613 (CD1) of chain 0 residue 169 (TYR)>
<Atom 2614 (HD1) of chain 0 residue 169 (TYR)>
<Atom 2615 (CE1) of chain 0 residue 169 (TYR)>
<Atom 2616 (HE1) of chain 0 residue 169 (TYR)>
<Atom 2617 (CZ) of chain 0 residue 169 (TYR)>
<Atom 2618 (OH) of chain 0 residue 169 (TYR)>
<Atom 2619 (HH) of chain 0 residue 169 (TYR)>
<Atom 2620 (CE2) of chain 0 residue 169 (TYR)>
<Atom 2621 (HE2) of chain 0 residue 169 (TYR)>
<Atom 2622 (CD2) of chain 0 residue 169 (TYR)>
<Atom 2623 (HD2) of chain 0 residue 169 (TYR)>
<Atom 2624 (C) of chain 0 residue 169 (TYR)>
<Atom 2625 (O) of chain 0 residue 169 (TYR)>
<Atom 3920 (O) of chain 2 re

Using htf.new_positions(), htf.old_positions()

In [9]:
dihedrals, n_iter, all_pos_hybrid = get_dihedrals(i, name, length, outdir, htf, indices_new, indices_old)


  0%|          | 0/10 [00:00<?, ?it/s]

first solvent atom  15974


100%|██████████| 10/10 [04:19<00:00, 25.98s/it]

get time 0.9887659549713135
old time: 128.98917198181152
new time: 129.80154728889465
hybrid time: 0.012508153915405273


Using faster version

In [39]:
dihedrals, n_iter, all_pos_hybrid = get_dihedrals(i, name, length, outdir, htf, indices_new, indices_old)


 20%|██        | 2/10 [00:00<00:00, 16.54it/s]

shape  (58648, 3)
shape  (58648, 3)
shape  (58648, 3)
shape  (58648, 3)
shape  (58648, 3)
shape  (58648, 3)
shape  (58648, 3)
shape  (58648, 3)


 60%|██████    | 6/10 [00:00<00:00, 14.47it/s]

shape  (58648, 3)
shape  (58648, 3)
shape  (58648, 3)
shape  (58648, 3)
shape  (58648, 3)
shape  (58648, 3)


 80%|████████  | 8/10 [00:00<00:00, 11.27it/s]

shape  (58648, 3)
shape  (58648, 3)
shape  (58648, 3)
shape  (58648, 3)
shape  (58648, 3)
shape  (58648, 3)


100%|██████████| 10/10 [00:00<00:00, 13.21it/s]


# Image the traj

In [52]:
traj = get_dihedrals(i, name, length, outdir, htf, indices_new, indices_old)


100%|██████████| 1/1 [00:26<00:00, 26.00s/it]

get time 0.0020825862884521484
old time: 12.93757438659668
new time: 13.059504985809326
hybrid time: 0.0010159015655517578


In [48]:
box_vectors = np.array([val.value_in_unit_system(unit.md_unit_system) for val in htf.hybrid_system.getDefaultPeriodicBoxVectors()])
htf._topology_proposal.old_topology.setPeriodicBoxVectors(box_vectors)
htf._topology_proposal.old_topology.getPeriodicBoxVectors()


Quantity(value=array([[11.4609322,  0.       ,  0.       ],
       [ 0.       , 11.8869192,  0.       ],
       [ 0.       ,  0.       , 17.003709 ]]), unit=nanometer)

In [62]:
vectors = np.array([htf._topology_proposal.old_topology.getPeriodicBoxVectors().value_in_unit_system(unit.md_unit_system)]) 

In [65]:
# Set unit cell vectors in traj 
traj.unitcell_vectors = np.repeat(vectors, 1, axis=0)
traj.image_molecules()


<mdtraj.Trajectory with 1 frames, 200474 atoms, 62711 residues, and unitcells at 0x2aff68a5bbb0>

In [66]:
traj.save("test_old.dcd")
traj[0].save("test_old.pdb")

## Draw 100 samples without replacement

In [68]:
data = list(range(200))
samples = random.sample(data, k=100) 
len(set(samples))

100

In [70]:
data = list(range(200))
samples = random.choices(data, k=100) 
len(set(samples))

84

## Fix THR/ILE indices

In [1]:
import math
from simtk import unit
import os
import tempfile
import pickle
import mdtraj as md
import numpy as np
from simtk.unit.quantity import Quantity
import logging 

# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

from matplotlib import pyplot as plt
from simtk.openmm import app
from tqdm import tqdm, tqdm_notebook
import argparse
import random
import time


In [2]:
def get_dihedrals(i, name, length, out_dir, htf, dihedral_indices_new, dihedral_indices_old):
    
    from tqdm import tqdm
    
    old_top = md.Topology.from_openmm(htf._topology_proposal.old_topology)
    new_top = md.Topology.from_openmm(htf._topology_proposal.new_topology)
    
    # Load nc file
    # From Hannah: https://github.com/hannahbrucemacdonald/endstate_pdbs/blob/master/scripts/input_for_pol_calc.py
    from perses.analysis.utils import open_netcdf
    nc = open_netcdf(os.path.join(out_dir, f"{i}_{phase}_{name.lower()}_{length}ns.nc"))
    nc_checkpoint = open_netcdf(os.path.join(out_dir, f"{i}_{phase}_{name.lower()}_{length}ns_checkpoint.nc"))
    checkpoint_interval = nc_checkpoint.CheckpointInterval
    all_positions = nc_checkpoint.variables['positions']
    n_iter, n_replicas, n_atoms, _ = np.shape(all_positions)
    n_iter = 1

    # Initialize arrays for old, new, and hybrid positions
    all_pos_new = np.zeros(shape=(n_iter, new_top.n_atoms, 3))
    all_pos_old = np.zeros(shape=(n_iter, old_top.n_atoms, 3))
    all_pos_hybrid = np.zeros(shape=(n_iter, n_atoms, 3)) # includes solvent atoms
    
    def new_positions(hybrid_positions):
        n_atoms_new = htf._topology_proposal.n_atoms_new
        hybrid_indices = [htf._new_to_hybrid_map[idx] for idx in range(n_atoms_new)]
        return hybrid_positions[hybrid_indices, :]
    
    def old_positions(hybrid_positions):
        n_atoms_old = htf._topology_proposal.n_atoms_old
        hybrid_indices = [htf._old_to_hybrid_map[idx] for idx in range(n_atoms_old)]
        return hybrid_positions[hybrid_indices, :]
    
    # Save old, new, and hybrid positions
    get_time = 0
    old_time = 0
    new_time = 0
    hybrid_time = 0
    for iteration in tqdm(range(n_iter)):
        initial_time = time.time()
        replica_id = np.where(nc.variables['states'][iteration*checkpoint_interval] == 0)[0]
        pos = all_positions[iteration,replica_id,:,:][0] *unit.nanometers
        elapsed_time = (time.time() - initial_time)
        get_time += elapsed_time

        initial_time = time.time()
        all_pos_new[iteration] = new_positions(pos).value_in_unit_system(unit.md_unit_system) 
        elapsed_time = (time.time() - initial_time)
        new_time += elapsed_time
        
        initial_time = time.time()
        all_pos_hybrid[iteration] = pos 
        elapsed_time = (time.time() - initial_time)
        hybrid_time += elapsed_time
        
        initial_time = time.time()
        all_pos_old[iteration] = old_positions(pos).value_in_unit_system(unit.md_unit_system)
        elapsed_time = (time.time() - initial_time)
        old_time += elapsed_time
    
    print(f"get time {get_time}") 
    print(f"old time: {old_time}")
    print(f"new time: {new_time}")
    print(f"hybrid time: {hybrid_time}")
    
    dihedrals_all = []
    for pos, top, indices in zip([all_pos_new, all_pos_old], [new_top, old_top], [dihedral_indices_new, dihedral_indices_old]):
        traj = md.Trajectory(np.array(pos), top)
        dihedrals = md.compute_dihedrals(traj, np.array([indices]))
        dihedrals_all.append(dihedrals)
    
    return traj
#     traj = traj.image_molecules()
    
#     traj.save("test_old.dcd")
#     traj[0].save("test_old.pdb")
    
#     return dihedrals_all, n_iter, all_pos_hybrid
    
def plot_dihedrals(dihedrals, outfile):
    # Plot histogram with error bars : https://stackoverflow.com/questions/35390276/how-to-add-error-bars-to-histogram-diagram-in-python
    entries, edges, _ = plt.hist(dihedrals)
    bin_centers = 0.5 * (edges[:-1] + edges[1:]) # calculate bin centers
    plt.errorbar(bin_centers, entries, yerr=np.sqrt(entries), fmt='r.') # draw errobars, use the sqrt error
    plt.xlim(-np.pi, np.pi)
    plt.savefig(outfile, dpi=300)
    plt.close()
    
def plot_time_series(dihedrals, n_iter, outfile):
    from perses.dispersed import feptasks
    t0, g, neff_max, a_t, uncorrelated_indices = feptasks.compute_timeseries(dihedrals)

    plt.scatter(range(n_iter), dihedrals)
    plt.ylabel("dihedral")
    plt.xlabel("iteration number")
    plt.ylim(-np.pi, np.pi)
    plt.savefig(outfile, dpi=300)
    plt.close()
    
    return uncorrelated_indices
    
def plot_dihedrals_uncorrelated(dihedrals, uncorrelated_indices, outfile):
    # Plot histogram with error bars : https://stackoverflow.com/questions/35390276/how-to-add-error-bars-to-histogram-diagram-in-python
    entries, edges, _ = plt.hist(dihedrals[uncorrelated_indices])
    bin_centers = 0.5 * (edges[:-1] + edges[1:]) # calculate bin centers
    plt.errorbar(bin_centers, entries, yerr=np.sqrt(entries), fmt='r.') # draw errobars, use the sqrt error
    plt.xlim(-np.pi, np.pi)
    plt.savefig(outfile, dpi=300)
    plt.close()

In [7]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/14/5/"
endstate = 1
phase = "complex"
length = 1
i = 5
old_aa_name = 'THR'
new_aa_name = 'ILE'
resid = '500'
name = 'ile'

In [5]:
with open(os.path.join(outdir, f"{i}_{phase}_{endstate}.pickle"), 'rb') as f:
    htf = pickle.load(f)


DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=6)
           2	LOAD_GLOBAL(arg=0, lineno=6)
           4	LOAD_METHOD(arg=1, lineno=6)
           6	LOAD_CONST(arg=1, lineno=6)
           8	CALL_METHOD(arg=1, lineno=6)
          10	STORE_FAST(arg=2, lineno=6)
          12	LOAD_FAST(arg=0, lineno=7)
          14	LOAD_CONST(arg=2, lineno=7)
          16	BINARY_SUBSCR(arg=None, lineno=7)
          18	LOAD_FAST(arg=1, lineno=7)
          20	LOAD_CONST(arg=3, lineno=7)
          22	BINARY_SUBSCR(arg=None, lineno=7)
          24	BINARY_MULTIPLY(arg=None, lineno=7)
          26	LOAD_FAST(arg=0, lineno=7)
          28	LOAD_CONST(arg=3, lineno=7)
          30	BINARY_SUBSCR(arg=None, lineno=7)
          32	LOAD_FAST(arg=1, lineno=7)
          34	LOAD_CONST(arg=2, lineno=7)
          36	BINARY_SUBSCR(arg=None, lineno=7)
          38	BINARY_MULTIPLY(arg=None, lineno=7)
          40	BINARY_SUBTRACT(arg=None, lineno=7)
          42	LOAD_FAST(arg=2, lineno=7)
          44	LO

DEBUG:numba.core.byteflow:dispatch pc=74, inst=BINARY_MULTIPLY(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$66binary_subscr.30', '$72binary_subscr.33']
DEBUG:numba.core.byteflow:dispatch pc=76, inst=BINARY_SUBTRACT(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$74binary_multiply.34']
DEBUG:numba.core.byteflow:dispatch pc=78, inst=LOAD_FAST(arg=2, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35']
DEBUG:numba.core.byteflow:dispatch pc=80, inst=LOAD_CONST(arg=2, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35', '$c78.36']
DEBUG:numba.core.byteflow:dispatch pc=82, inst=STORE_SUBSCR(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35', '$c78.36', '$const80.37']
DEBUG:numba.core.byteflow:dispatch pc=84, inst=LOAD_FAST(arg=0, lineno=9)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=86, inst=LOAD_CONST(arg=4, lineno=9)
DEBUG:numba.core.bytefl

DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x2b853a8fc700>
DEBUG:numba.core.ssa:on stmt: a = arg(0, name=a)
DEBUG:numba.core.ssa:on stmt: b = arg(1, name=b)
DEBUG:numba.core.ssa:on stmt: $2load_global.0 = global(np: <module 'numpy' from '/home/zhangi/miniconda3/envs/perses-rbd-ace2/lib/python3.8/site-packages/numpy/__init__.py'>)
DEBUG:numba.core.ssa:on stmt: $4load_method.1 = getattr(value=$2load_global.0, attr=zeros)
DEBUG:numba.core.ssa:on stmt: $const6.2 = const(int, 3)
DEBUG:numba.core.ssa:on stmt: $8call_method.3 = call $4load_method.1($const6.2, func=$4load_method.1, args=[Var($const6.2, coordinate_numba.py:6)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: c = $8call_method.3
DEBUG:numba.core.ssa:on stmt: $const14.5 = const(int, 1)
DEBUG:numba.core.ssa:on stmt: $16binary_subscr.6 = static_getitem(value=a, index=1, index_var=$const14.5, fn=<built-in function getitem>)
DEBUG:numba.core.ssa:

DEBUG:numba.core.ssa:SSA violators set()
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=14)
           2	LOAD_GLOBAL(arg=0, lineno=14)
           4	LOAD_METHOD(arg=1, lineno=14)
           6	LOAD_FAST(arg=0, lineno=14)
           8	LOAD_FAST(arg=0, lineno=14)
          10	CALL_METHOD(arg=2, lineno=14)
          12	STORE_FAST(arg=1, lineno=14)
          14	LOAD_GLOBAL(arg=0, lineno=15)
          16	LOAD_METHOD(arg=2, lineno=15)
          18	LOAD_FAST(arg=1, lineno=15)
          20	CALL_METHOD(arg=1, lineno=15)
          22	RETURN_VALUE(arg=None, lineno=15)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=14)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=2, inst=LOAD_GLOBAL(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=4, inst=LOAD_METHOD(arg=1, lineno=14)


DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=508)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=2, inst=LOAD_FAST(arg=0, lineno=508)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=4, inst=LOAD_ATTR(arg=0, lineno=508)
DEBUG:numba.core.byteflow:stack ['$a2.0']
DEBUG:numba.core.byteflow:dispatch pc=6, inst=UNPACK_SEQUENCE(arg=1, lineno=508)
DEBUG:numba.core.byteflow:stack ['$4load_attr.1']
DEBUG:numba.core.byteflow:dispatch pc=8, inst=STORE_FAST(arg=2, lineno=508)
DEBUG:numba.core.byteflow:stack ['$6unpack_sequence.2']
DEBUG:numba.core.byteflow:dispatch pc=10, inst=LOAD_FAST(arg=1, lineno=509)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=12, inst=LOAD_ATTR(arg=0, lineno=509)
DEBUG:numba.core.byteflow:stack ['$b10.4']
DEBUG:numba.core.byteflow:dispatch pc=14, inst=UNPACK_SEQUENCE(arg=1, lineno=509)
DEBUG:numba.core.byteflow:stack ['$12load_attr.5']
DEBU

...

DEBUG:numba.core.ssa:on stmt: $154load_global.20 = global(_norm: CPUDispatcher(<function _norm at 0x2b853aa51c10>))
DEBUG:numba.core.ssa:on stmt: $158call_function.22 = call $154load_global.20(plane2, func=$154load_global.20, args=[Var(plane2, coordinate_numba.py:127)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $160binary_multiply.23 = $152call_function.19 * $158call_function.22
DEBUG:numba.core.ssa:on stmt: $162binary_true_divide.24 = $146call_method.16 / $160binary_multiply.23
DEBUG:numba.core.ssa:on stmt: cos_phi = $162binary_true_divide.24
DEBUG:numba.core.ssa:on stmt: $const168.26 = const(float, -1.0)
DEBUG:numba.core.ssa:on stmt: $170compare_op.27 = cos_phi < $const168.26
DEBUG:numba.core.ssa:on stmt: bool172 = global(bool: <class 'bool'>)
DEBUG:numba.core.ssa:on stmt: $172pred = call bool172($170compare_op.27, func=bool172, args=(Var($170compare_op.27, coordinate_numba.py:130),), kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: branch $172pred, 174, 180
DEBUG:numba.co

DEBUG:numba.core.ssa:on stmt: $72load_method.28 = getattr(value=$70load_global.27, attr=dot)
DEBUG:numba.core.ssa:on stmt: $78call_method.31 = call $72load_method.28(a_u, b_u, func=$72load_method.28, args=[Var(a_u, coordinate_numba.py:110), Var(b_u, coordinate_numba.py:111)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: cos_theta = $78call_method.31
DEBUG:numba.core.ssa:on stmt: $const84.33 = const(float, 1.0)
DEBUG:numba.core.ssa:on stmt: $86compare_op.34 = cos_theta > $const84.33
DEBUG:numba.core.ssa:find_def var='cos_theta' stmt=$86compare_op.34 = cos_theta > $const84.33
DEBUG:numba.core.ssa:on stmt: bool88 = global(bool: <class 'bool'>)
DEBUG:numba.core.ssa:on stmt: $88pred = call bool88($86compare_op.34, func=bool88, args=(Var($86compare_op.34, coordinate_numba.py:119),), kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: branch $88pred, 90, 96
DEBUG:numba.core.ssa:==== SSA block rewrite pass on 90
DEBUG:numba.core.ssa:Running <numba.core.ssa._FixSSAVars object at 0x2b853b3d

DEBUG:numba.core.ssa:on stmt: $198call_method.3 = call $194load_method.1(cos_phi.3, func=$194load_method.1, args=[Var(cos_phi.3, coordinate_numba.py:135)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: phi = $198call_method.3
DEBUG:numba.core.ssa:on stmt: $202load_global.4 = global(np: <module 'numpy' from '/home/zhangi/miniconda3/envs/perses-rbd-ace2/lib/python3.8/site-packages/numpy/__init__.py'>)
DEBUG:numba.core.ssa:on stmt: $204load_method.5 = getattr(value=$202load_global.4, attr=dot)
DEBUG:numba.core.ssa:on stmt: $210call_method.8 = call $204load_method.5(a, plane2, func=$204load_method.5, args=[Var(a, coordinate_numba.py:106), Var(plane2, coordinate_numba.py:127)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $const212.9 = const(int, 0)
DEBUG:numba.core.ssa:on stmt: $214compare_op.10 = $210call_method.8 <= $const212.9
DEBUG:numba.core.ssa:on stmt: bool216 = global(bool: <class 'bool'>)
DEBUG:numba.core.ssa:on stmt: $216pred = call bool216($214compare_op.10, func=bool

DEBUG:numba.core.byteflow:----------------------DONE Prune PHIs-----------------------
DEBUG:numba.core.byteflow:block_infos State(pc_initial=0 nstack_initial=0):
AdaptBlockInfo(insts=((0, {}), (2, {'res': '$angle_position_12.0'}), (4, {'res': '$bond_position_24.1'}), (6, {'lhs': '$angle_position_12.0', 'rhs': '$bond_position_24.1', 'res': '$6binary_subtract.2'}), (8, {'retval': '$6binary_subtract.2', 'castval': '$8return_value.3'})), outgoing_phis={}, blockstack=(), active_try_block=None, outgoing_edgepushed={})
DEBUG:numba.core.interpreter:label 0:
    angle_position_1 = arg(0, name=angle_position_1) ['angle_position_1']
    bond_position_2 = arg(1, name=bond_position_2) ['bond_position_2']
    $6binary_subtract.2 = angle_position_1 - bond_position_2 ['$6binary_subtract.2', 'angle_position_1', 'bond_position_2']
    $8return_value.3 = cast(value=$6binary_subtract.2) ['$6binary_subtract.2', '$8return_value.3']
    return $8return_value.3                  ['$8return_value.3']

DEBUG:nu

DEBUG:numba.core.ssa:on stmt: a_1 = arg(1, name=a_1)
DEBUG:numba.core.ssa:on stmt: $6binary_true_divide.2 = a_1 / _32call_function_12_1
DEBUG:numba.core.ssa:on stmt: $8return_value.3 = cast(value=$6binary_true_divide.2)
DEBUG:numba.core.ssa:on stmt: return $8return_value.3
DEBUG:numba.core.ssa:defs defaultdict(<class 'list'>,
            {'$6binary_true_divide.2': [<numba.core.ir.Assign object at 0x2b853af4db80>],
             '$8return_value.3': [<numba.core.ir.Assign object at 0x2b853af4daf0>],
             '_32call_function_12_1': [<numba.core.ir.Assign object at 0x2b853af0ea90>],
             'a_1': [<numba.core.ir.Assign object at 0x2b853af4d340>]})
DEBUG:numba.core.ssa:SSA violators set()
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=107)
           2	LOAD_FAST(arg=1, lineno=107)
           4	LOAD_FAST(arg=0, lineno=111)
           6	BINARY_TRUE_DIVIDE(arg=None, lineno=1)
           8	RETURN_VALUE(arg=None, lineno=1)
DEBUG:numba.core.byteflow:pending:

In [15]:
thr_dihedral = ['N', 'CA', 'CB', 'OG1']
other_dihedral = ['N', 'CA', 'CB', 'CG']
ala_dihedral = ['N', 'CA', 'CB', 'HB1']
asp_dihedral = ['CA', 'CB', 'CG', 'OD2']
ile_dihedral = ['N', 'CA', 'CB', 'CG2']

dihedral_atoms = []
for aa_name in [old_aa_name, new_aa_name]:
    if aa_name in ["PHE", "TYR", "TRP", "GLU", "LYS", "ARG", "GLN", "ASN"]:
        dihedral_atoms.append(other_dihedral)
    elif aa_name == "THR":
        dihedral_atoms.append(thr_dihedral)
    elif aa_name == "ALA":
        dihedral_atoms.append(ala_dihedral)
    elif aa_name == 'ASP':
        dihedral_atoms.append(asp_dihedral)
    elif aa_name == 'ILE':
        dihedral_atoms.append(ile_dihedral)
        

for res in htf._topology_proposal.old_topology.residues():
    if res.id == resid and res.chain.index == 0:
        residue_old = res
for res in htf._topology_proposal.new_topology.residues():
    if res.id == resid and res.chain.index == 0:
        residue_new = res
# indices_old = [atom.index for atom in list(htf._topology_proposal.old_topology.residues())[int(args.resid)].atoms() if atom.name in dihedral_atoms[0]]
# indices_new = [atom.index for atom in list(htf._topology_proposal.new_topology.residues())[int(args.resid)].atoms() if atom.name in dihedral_atoms[1]]
indices_old = [atom.index for atom in residue_old.atoms() if atom.name in dihedral_atoms[0]]
indices_new = [atom.index for atom in residue_new.atoms() if atom.name in dihedral_atoms[1]]
_logger.info(f"old indices: {indices_old}")
_logger.info(f"new indices: {indices_new}")



INFO:root:old indices: [2591, 2593, 2595, 2601]
INFO:root:new indices: [2591, 2593, 2595, 2597]


In [10]:
dihedral_atoms

[['N', 'CA', 'CB', 'OG'], ['N', 'CA', 'CB', 'CG']]

In [17]:
for atom in htf._topology_proposal.old_topology.atoms():
    if atom.residue.name == 'THR' and atom.residue.id == '500':
        print(atom)

<Atom 2591 (N) of chain 0 residue 168 (THR)>
<Atom 2592 (H) of chain 0 residue 168 (THR)>
<Atom 2593 (CA) of chain 0 residue 168 (THR)>
<Atom 2594 (HA) of chain 0 residue 168 (THR)>
<Atom 2595 (CB) of chain 0 residue 168 (THR)>
<Atom 2596 (HB) of chain 0 residue 168 (THR)>
<Atom 2597 (CG2) of chain 0 residue 168 (THR)>
<Atom 2598 (HG21) of chain 0 residue 168 (THR)>
<Atom 2599 (HG22) of chain 0 residue 168 (THR)>
<Atom 2600 (HG23) of chain 0 residue 168 (THR)>
<Atom 2601 (OG1) of chain 0 residue 168 (THR)>
<Atom 2602 (HG1) of chain 0 residue 168 (THR)>
<Atom 2603 (C) of chain 0 residue 168 (THR)>
<Atom 2604 (O) of chain 0 residue 168 (THR)>


In [13]:
for atom in htf._topology_proposal.new_topology.atoms():
    if atom.residue.name == 'ILE' and atom.residue.id == '500':
        print(atom)

<Atom 2591 (N) of chain 0 residue 168 (ILE)>
<Atom 2592 (H) of chain 0 residue 168 (ILE)>
<Atom 2593 (CA) of chain 0 residue 168 (ILE)>
<Atom 2594 (HA) of chain 0 residue 168 (ILE)>
<Atom 2595 (CB) of chain 0 residue 168 (ILE)>
<Atom 2596 (HB) of chain 0 residue 168 (ILE)>
<Atom 2597 (CG2) of chain 0 residue 168 (ILE)>
<Atom 2598 (HG21) of chain 0 residue 168 (ILE)>
<Atom 2599 (HG22) of chain 0 residue 168 (ILE)>
<Atom 2600 (HG23) of chain 0 residue 168 (ILE)>
<Atom 2601 (CG1) of chain 0 residue 168 (ILE)>
<Atom 2602 (HG12) of chain 0 residue 168 (ILE)>
<Atom 2603 (HG13) of chain 0 residue 168 (ILE)>
<Atom 2604 (CD1) of chain 0 residue 168 (ILE)>
<Atom 2605 (HD11) of chain 0 residue 168 (ILE)>
<Atom 2606 (HD12) of chain 0 residue 168 (ILE)>
<Atom 2607 (HD13) of chain 0 residue 168 (ILE)>
<Atom 2608 (C) of chain 0 residue 168 (ILE)>
<Atom 2609 (O) of chain 0 residue 168 (ILE)>
